In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

import pandas as pd
pd.options.display.max_columns = 100

from matplotlib import pyplot as plt
import numpy as np

import seaborn as sns
sns.set(rc={'figure.figsize':(12,9)})
import pylab as plot
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,accuracy_score
import category_encoders as ce

In [2]:
from xgboost import XGBClassifier
import xgboost as xgb


In [3]:
#for scaling
from sklearn.preprocessing import StandardScaler

Importing training dataset

In [4]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
print(data.shape)

(867873, 10)


In [6]:
data.columns

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'Id'],
      dtype='object')

Splitting the "Dates" into date and time

In [7]:
df=data.head(25000)
datetime=df.Dates.str.split(pat=" ",expand=True)
datetime.columns=['Date','Time']
#datetime

#------------test data-------------

df_test=test
datetime_test=df_test.Dates.str.split(pat=" ",expand=True)
datetime_test.columns=['Date','Time']

In [8]:
Date=datetime.Date.str.split(pat="-",expand=True)
Date.columns=['Year','Month','Day']

Time=datetime.Time.str.split(pat=":",expand=True)
Time.columns=['Hour','Minute','Second']

#------------test data-------------

Date_test=datetime_test.Date.str.split(pat="-",expand=True)
Date_test.columns=['Year','Month','Day']

Time_test=datetime_test.Time.str.split(pat=":",expand=True)
Time_test.columns=['Hour','Minute','Second']

Dropping the column of "Dates" and appending the dataframe "datetime"

In [9]:
df=pd.concat([df,Date,Time],axis=1)
#df

#-----------test data------------------

df_test=pd.concat([df_test,Date_test,Time_test],axis=1)


In [10]:
df=df.drop(labels=['Dates'],axis=1)

#-----------test data------------------

df_test=df_test.drop(labels=['Dates'],axis=1)


# #Encoding

In [11]:
df.columns

Index(['Category', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution',
       'Address', 'X', 'Y', 'Id', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Second'],
      dtype='object')

LabelEncoding of "DayOfWeek" and "PdDistrict"

In [12]:
le = preprocessing.LabelEncoder()

In [13]:
#y=pd.get_dummies(df.Category,columns=['Category'],prefix=" ",prefix_sep=" ",drop_first=True,)

le_res=le.fit_transform(df['Category'])
y=pd.DataFrame(le_res)
y.columns=['Category']
#y

# Degree to Radian

In [14]:
df['Y']=df['Y'].apply(lambda x : x*0.0174533 )
df['X']=df['X'].apply(lambda x : x*0.0174533)

# Rotation 45 degree 

In [15]:
#df['Y']=df['Y'].apply(lambda x : x if 37.82 > x else 37.82)
#df['X']=df['X'].apply(lambda x : x if -122.3 > x else 122.3)

#--------test data--------------

#df_test['Y']=df_test['Y'].apply(lambda x : x if 37.82 > x else 37.82)
#df_test['X']=df_test['X'].apply(lambda x : x if -122.3 > x else 122.3)

In [16]:
df["rot45_X"]=0.707 * df["Y"] + 0.707 * df["X"]
df["rot45_Y"]=0.707 * df["Y"] - 0.707 * df["X"]

#----------test data------------

df_test["rot45_X"]=0.707 * df_test["Y"] + 0.707 * df_test["X"]
df_test["rot45_Y"]=0.707 * df_test["Y"] - 0.707 * df_test["X"]

In [17]:
df["radial45"]=np.sqrt(np.power(df['rot45_X'],2) + np.power(df['rot45_Y'],2))

#-----------test data------------

df_test["radial45"]=np.sqrt(np.power(df_test['rot45_X'],2) + np.power(df_test['rot45_Y'],2))

# Rotation 30 degree

In [18]:
df["rot30_X"]=(1.732/2) * df["Y"] + 0.5 * df["X"]
df["rot30_Y"]=(1.732/2) * df["Y"] - 0.5 * df["X"]

df_test["rot30_X"]=(1.732/2) * df_test["Y"] + 0.5 * df_test["X"]
df_test["rot30_Y"]=(1.732/2) * df_test["Y"] - 0.5 * df_test["X"]

In [ ]:
df["radial30"]=np.sqrt(np.power(df['rot30_X'],2) + np.power(df['rot30_Y'],2))

df_test["radial30"]=np.sqrt(np.power(df_test['rot30_X'],2) + np.power(df_test['rot30_Y'],2))

# Rotation 60 degree

In [ ]:
df["rot60_X"]=(0.5) * df["Y"] + (1.732/2) * df["X"]
df["rot60_Y"]=0.5 * df["Y"] - (1.732/2) * df["X"]

df_test["rot60_X"]=(0.5) * df_test["Y"] + (1.732/2) * df_test["X"]
df_test["rot60_Y"]=0.5 * df_test["Y"] - (1.732/2) * df_test["X"]

In [ ]:
df["radial60"]=np.sqrt(np.power(df['rot60_X'],2) + np.power(df['rot60_Y'],2))

df_test["radial60"]=np.sqrt(np.power(df_test['rot60_X'],2) + np.power(df_test['rot60_Y'],2))

# Hours clubbing

In [ ]:
df['Hour']=df['Hour'].apply(lambda x:int(x))
#df['Hour']=df['Hour'].apply(lambda x : 'EARLY_Morning' if (x >= 1) and (x <=5) else ('MORNING' if x >= 6 and x <=11 else ('DAY' if x>=12 and x<=18 else 'Night')))
df['Hour']=df['Hour'].apply(lambda x : 'EARLY_MORNING' if x in (1,2,3,4,5) else ('MORNING' if x in (6,7,8,9,10,11) else ('DAY' if x in (12,13,14,15,16,17,18) else 'NIGHT')))

df_test['Hour']=df_test['Hour'].apply(lambda x:int(x))
#df['Hour']=df['Hour'].apply(lambda x : 'EARLY_Morning' if (x >= 1) and (x <=5) else ('MORNING' if x >= 6 and x <=11 else ('DAY' if x>=12 and x<=18 else 'Night')))
df_test['Hour']=df_test['Hour'].apply(lambda x : 'EARLY_MORNING' if x in (1,2,3,4,5) else ('MORNING' if x in (6,7,8,9,10,11) else ('DAY' if x in (12,13,14,15,16,17,18) else 'NIGHT')))

# Month Clubbing

In [ ]:
df['Month']=df['Month'].apply(lambda x:int(x))
#df['Month']=df['Month'].apply(lambda x : 'MonthLow' if x== 12 else ('MonthMed' if x in (2,6,7,8,9,11) else 'MonthHigh'))
df['Month']=df['Month'].apply(lambda x : 'Winter' if x in (11,12,1,2) else ('Spring' if x in (3,4,5) else('Summer' if x in (6,7,8) else('Fall'))))

df_test['Month']=df_test['Month'].apply(lambda x:int(x))
#df['Month']=df['Month'].apply(lambda x : 'MonthLow' if x== 12 else ('MonthMed' if x in (2,6,7,8,9,11) else 'MonthHigh'))
df_test['Month']=df_test['Month'].apply(lambda x : 'Winter' if x in (11,12,1,2) else ('Spring' if x in (3,4,5) else('Summer' if x in (6,7,8) else('Fall'))))

# Minute Clubbing

In [ ]:
df['Minute']=df['Minute'].apply(lambda x:int(x))
df['Minute']=df['Minute'].apply(lambda x : 'low' if x <31 else 'high')

df_test['Minute']=df_test['Minute'].apply(lambda x:int(x))
df_test['Minute']=df_test['Minute'].apply(lambda x : 'low' if x <31 else 'high')

# DayOfWeek

In [ ]:
df['DayOfWeek']= df['DayOfWeek'].apply(lambda x : 'WeekHigh' if x in (3,5) else ('WeekMed' if x in (2,4,6) else 'WeekLow'))

df_test['DayOfWeek']= df_test['DayOfWeek'].apply(lambda x : 'WeekHigh' if x in (3,5) else ('WeekMed' if x in (2,4,6) else 'WeekLow'))


# Address Update

In [ ]:
df['Intersection']=df['Address'].apply(lambda x : 1 if '/' in x else 0)
df['Block']=df['Address'].apply(lambda x : 1 if 'Block' in x else 0)
#df['StreetSuffix']=df['Address'].apply(lambda x : x.split(' ')[-1] if len(x.split(' ')[-1])==2 else 0) 

df_test['Intersection']=df_test['Address'].apply(lambda x : 1 if '/' in x else 0)
df_test['Block']=df_test['Address'].apply(lambda x : 1 if 'Block' in x else 0)
#df_test['StreetSuffix']=df_test['Address'].apply(lambda x : x.split(' ')[-1] if len(x.split(' ')[-1])==2 else 0) 

# Address splitting

In [ ]:
"""address=pd.DataFrame(df['Address'],columns=['Address'])
address=address.Address.str.split(pat=" /",expand=True )

address.columns=['Address','Intr2']

address=address.Address.str.split(pat=" /",expand=True )
address.columns=['Address']

#----------test data----------

address_test=pd.DataFrame(df_test['Address'],columns=['Address'])
address_test=address_test.Address.str.split(pat=" /",expand=True )

address_test.columns=['Address','Intr2']

address_test=address_test.Address.str.split(pat=" /",expand=True )
address_test.columns=['Address']
"""

'address=pd.DataFrame(df[\'Address\'],columns=[\'Address\'])\naddress=address.Address.str.split(pat=" /",expand=True )\n\naddress.columns=[\'Address\',\'Intr2\']\n\naddress=address.Address.str.split(pat=" /",expand=True )\naddress.columns=[\'Address\']\n\n#----------test data----------\n\naddress_test=pd.DataFrame(df_test[\'Address\'],columns=[\'Address\'])\naddress_test=address_test.Address.str.split(pat=" /",expand=True )\n\naddress_test.columns=[\'Address\',\'Intr2\']\n\naddress_test=address_test.Address.str.split(pat=" /",expand=True )\naddress_test.columns=[\'Address\']\n'

In [ ]:
"""string=address.iloc[:,0]
string=string.str.strip()

#----------test data----------

string_test=address_test.iloc[:,0]
string_test=string_test.str.strip()"""

'string=address.iloc[:,0]\nstring=string.str.strip()\n\n#----------test data----------\n\nstring_test=address_test.iloc[:,0]\nstring_test=string_test.str.strip()'

In [ ]:
"""address_fram=string.to_frame()
temp=address_fram['Address'].astype(str).str[-2:]

#----------test data----------
address_fram_test=string_test.to_frame()
temp_test=address_fram_test['Address'].astype(str).str[-2:]"""

"address_fram=string.to_frame()\ntemp=address_fram['Address'].astype(str).str[-2:]\n\n#----------test data----------\naddress_fram_test=string_test.to_frame()\ntemp_test=address_fram_test['Address'].astype(str).str[-2:]"

In [ ]:
"""address=temp.to_frame()

#----------test data----------

address_test=temp_test.to_frame()"""

'address=temp.to_frame()\n\n#----------test data----------\n\naddress_test=temp_test.to_frame()'

In [ ]:
"""address['Address']=address['Address'].apply(lambda x :( x if x in ("ST","AV","LN","DR","BL","HY","CT","RD","PL","PZ","TR","AL","CR","WK","EX","RW") else (("I-80" if x in ("80") else ("HWY" if x in ("WY") else ("WAY" if x in ("AY") else ("TER" if x in ("ER") else ("ALMS" if x in ("MS") else ("MAR" if x in ("AR") else ("PARK" if x in ("RK") else ("STWY" if x in ("WY") else ("VIA" if x in ("NO") else ("BLOCK")))))))))))))
#----------test data----------
address_test['Address']=address_test['Address'].apply(lambda x :( x if x in ("ST","AV","LN","DR","BL","HY","CT","RD","PL","PZ","TR","AL","CR","WK","EX","RW") else (("I-80" if x in ("80") else ("HWY" if x in ("WY") else ("WAY" if x in ("AY") else ("TER" if x in ("ER") else ("ALMS" if x in ("MS") else ("MAR" if x in ("AR") else ("PARK" if x in ("RK") else ("STWY" if x in ("WY") else ("VIA" if x in ("NO") else ("BLOCK")))))))))))))"""

'address[\'Address\']=address[\'Address\'].apply(lambda x :( x if x in ("ST","AV","LN","DR","BL","HY","CT","RD","PL","PZ","TR","AL","CR","WK","EX","RW") else (("I-80" if x in ("80") else ("HWY" if x in ("WY") else ("WAY" if x in ("AY") else ("TER" if x in ("ER") else ("ALMS" if x in ("MS") else ("MAR" if x in ("AR") else ("PARK" if x in ("RK") else ("STWY" if x in ("WY") else ("VIA" if x in ("NO") else ("BLOCK")))))))))))))\n#----------test data----------\naddress_test[\'Address\']=address_test[\'Address\'].apply(lambda x :( x if x in ("ST","AV","LN","DR","BL","HY","CT","RD","PL","PZ","TR","AL","CR","WK","EX","RW") else (("I-80" if x in ("80") else ("HWY" if x in ("WY") else ("WAY" if x in ("AY") else ("TER" if x in ("ER") else ("ALMS" if x in ("MS") else ("MAR" if x in ("AR") else ("PARK" if x in ("RK") else ("STWY" if x in ("WY") else ("VIA" if x in ("NO") else ("BLOCK")))))))))))))'

In [ ]:
"""df=df.drop(labels=['Address'],axis=1)
#----------test data----------
df_test=df_test.drop(labels=['Address'],axis=1)"""

"df=df.drop(labels=['Address'],axis=1)\n#----------test data----------\ndf_test=df_test.drop(labels=['Address'],axis=1)"

In [ ]:
"""df=pd.concat([address,df],axis=1)
#----------test data----------
df_test=pd.concat([address_test,df_test],axis=1)"""

'df=pd.concat([address,df],axis=1)\n#----------test data----------\ndf_test=pd.concat([address_test,df_test],axis=1)'

In [ ]:
Id=df['Id']
df=df.drop(['Descript','Category','Resolution','Id','Second'],axis=1)

#----------test data---------

Id_test=df_test['Id']
df_test=df_test.drop(['Descript','Resolution','Id','Second'],axis=1)

In [ ]:
df_test.head()

,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Day,Hour,Minute,rot45_X,rot45_Y,radial45,rot30_X,rot30_Y,radial30,rot60_X,rot60_Y,radial60,Intersection,Block
0,WeekLow,TENDERLOIN,100 Block of POWELL ST,-122.407878,37.785968,2010,Summer,04,NIGHT,low,-59.827690,113.257049,128.087906,-28.481291,93.926587,98.149823,-87.112238,124.898206,152.276406,0,1
1,WeekLow,PARK,2400 Block of GEARY BL,-122.443597,37.782644,2004,Summer,26,NIGHT,high,-59.855294,113.279952,128.121051,-28.502029,93.941568,98.170178,-87.144833,124.927477,152.319061,0,1
2,WeekLow,SOUTHERN,HOWARD ST / 3RD ST,-122.400474,37.785029,2013,Winter,09,NIGHT,low,-59.823119,113.251150,128.080555,-28.478402,93.922072,98.144663,-87.106296,124.891325,152.267363,1,0
3,WeekLow,NORTHERN,VANNESS AV / HAYES ST,-122.419698,37.777301,2006,Winter,03,NIGHT,low,-59.842175,113.259278,128.096643,-28.494706,93.924991,98.152190,-87.126808,124.904109,152.289583,1,0
4,WeekLow,PARK,BEULAH ST / SHRADER ST,-122.451488,37.767516,2014,Fall,21,MORNING,high,-59.871568,113.274836,128.124131,-28.519075,93.932413,98.166368,-87.159230,124.926746,152.326699,1,0


In [ ]:
data['PdDistrict'].unique()

array(['MISSION', 'SOUTHERN', 'BAYVIEW', 'CENTRAL', 'INGLESIDE',
       'NORTHERN', 'RICHMOND', 'TARAVAL', 'TENDERLOIN', 'PARK'],
      dtype=object)

# Label Encoding

In [ ]:
le = preprocessing.LabelEncoder()

Label Encoding :DayOfWeek

In [ ]:
"""
#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['DayOfWeek'])
Day=pd.DataFrame(le_res)
Day.columns=['DayOfWeek']
df=df.drop(labels=['DayOfWeek'],axis=1)
df=pd.concat([Day,df],axis=1)

#----------test data----------

le_res_test=le.fit_transform(df_test['DayOfWeek'])
Day_test=pd.DataFrame(le_res_test)
Day_test.columns=['DayOfWeek']
df_test=df_test.drop(labels=['DayOfWeek'],axis=1)
df_test=pd.concat([Day_test,df_test],axis=1)"""

"\n#le = preprocessing.LabelEncoder()\nle_res=le.fit_transform(df['DayOfWeek'])\nDay=pd.DataFrame(le_res)\nDay.columns=['DayOfWeek']\ndf=df.drop(labels=['DayOfWeek'],axis=1)\ndf=pd.concat([Day,df],axis=1)\n\n#----------test data----------\n\nle_res_test=le.fit_transform(df_test['DayOfWeek'])\nDay_test=pd.DataFrame(le_res_test)\nDay_test.columns=['DayOfWeek']\ndf_test=df_test.drop(labels=['DayOfWeek'],axis=1)\ndf_test=pd.concat([Day_test,df_test],axis=1)"

Label Encoding: PdDistrict

In [ ]:
"""
le_res=le.fit_transform(df['PdDistrict'])
District=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
District.columns=['District']
df=df.drop(labels=['PdDistrict'],axis=1)
df=pd.concat([District,df],axis=1)


le_res_test=le.fit_transform(df_test['PdDistrict'])
District_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
District_test.columns=['District']
df_test=df_test.drop(labels=['PdDistrict'],axis=1)
df_test=pd.concat([District_test,df_test],axis=1)
"""

"\nle_res=le.fit_transform(df['PdDistrict'])\nDistrict=pd.DataFrame(le_res)\n\n#District=pd.get_dummies(df['PdDistrict'],drop_first=True)\nDistrict.columns=['District']\ndf=df.drop(labels=['PdDistrict'],axis=1)\ndf=pd.concat([District,df],axis=1)\n\n\nle_res_test=le.fit_transform(df_test['PdDistrict'])\nDistrict_test=pd.DataFrame(le_res_test)\n\n#District=pd.get_dummies(df['PdDistrict'],drop_first=True)\nDistrict_test.columns=['District']\ndf_test=df_test.drop(labels=['PdDistrict'],axis=1)\ndf_test=pd.concat([District_test,df_test],axis=1)\n"

Label encoding : Address

In [ ]:

le_res=le.fit_transform(df['Address'])
Address=pd.DataFrame(le_res)
Address.columns=['Address']
df=df.drop(labels=['Address'],axis=1)
df=pd.concat([Address,df],axis=1)

le_res_test=le.fit_transform(df_test['Address'])
Address_test=pd.DataFrame(le_res_test)
Address_test.columns=['Address']
df_test=df_test.drop(labels=['Address'],axis=1)
df_test=pd.concat([Address_test,df_test],axis=1)


Label encoding: Year

In [ ]:
"""
#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Year'])
Year=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Year.columns=['Year']
df=df.drop(labels=['Year'],axis=1)
df=pd.concat([Year,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Year'])
Year_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Year_test.columns=['Year']
df_test=df_test.drop(labels=['Year'],axis=1)
df_test=pd.concat([Year_test,df_test],axis=1)

"""

"\n#le = preprocessing.LabelEncoder()\nle_res=le.fit_transform(df['Year'])\nYear=pd.DataFrame(le_res)\n\n#District=pd.get_dummies(df['PdDistrict'],drop_first=True)\nYear.columns=['Year']\ndf=df.drop(labels=['Year'],axis=1)\ndf=pd.concat([Year,df],axis=1)\n\n\n#le = preprocessing.LabelEncoder()\nle_res_test=le.fit_transform(df_test['Year'])\nYear_test=pd.DataFrame(le_res_test)\n\n#District=pd.get_dummies(df['PdDistrict'],drop_first=True)\nYear_test.columns=['Year']\ndf_test=df_test.drop(labels=['Year'],axis=1)\ndf_test=pd.concat([Year_test,df_test],axis=1)\n\n"

Label encoding: Month

In [ ]:
"""
#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Month'])
Month=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Month.columns=['Month']
df=df.drop(labels=['Month'],axis=1)
df=pd.concat([Month,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Month'])
Month_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Month_test.columns=['Month']
df_test=df_test.drop(labels=['Month'],axis=1)
df_test=pd.concat([Month_test,df_test],axis=1)
"""

"\n#le = preprocessing.LabelEncoder()\nle_res=le.fit_transform(df['Month'])\nMonth=pd.DataFrame(le_res)\n\n#District=pd.get_dummies(df['PdDistrict'],drop_first=True)\nMonth.columns=['Month']\ndf=df.drop(labels=['Month'],axis=1)\ndf=pd.concat([Month,df],axis=1)\n\n\n#le = preprocessing.LabelEncoder()\nle_res_test=le.fit_transform(df_test['Month'])\nMonth_test=pd.DataFrame(le_res_test)\n\n#District=pd.get_dummies(df['PdDistrict'],drop_first=True)\nMonth_test.columns=['Month']\ndf_test=df_test.drop(labels=['Month'],axis=1)\ndf_test=pd.concat([Month_test,df_test],axis=1)\n"

Label encoding: Day

In [ ]:
"""
#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Day'])
Day=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Day.columns=['Day']
df=df.drop(labels=['Day'],axis=1)
df=pd.concat([Day,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Day'])
Day_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Day_test.columns=['Day']
df_test=df_test.drop(labels=['Day'],axis=1)
df_test=pd.concat([Day_test,df_test],axis=1)
"""

"\n#le = preprocessing.LabelEncoder()\nle_res=le.fit_transform(df['Day'])\nDay=pd.DataFrame(le_res)\n\n#District=pd.get_dummies(df['PdDistrict'],drop_first=True)\nDay.columns=['Day']\ndf=df.drop(labels=['Day'],axis=1)\ndf=pd.concat([Day,df],axis=1)\n\n\n#le = preprocessing.LabelEncoder()\nle_res_test=le.fit_transform(df_test['Day'])\nDay_test=pd.DataFrame(le_res_test)\n\n#District=pd.get_dummies(df['PdDistrict'],drop_first=True)\nDay_test.columns=['Day']\ndf_test=df_test.drop(labels=['Day'],axis=1)\ndf_test=pd.concat([Day_test,df_test],axis=1)\n"

Label encoding: Hour

In [ ]:
"""
#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Hour'])
Hour=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Hour.columns=['Hour']
df=df.drop(labels=['Hour'],axis=1)
df=pd.concat([Hour,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Hour'])
Hour_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Hour_test.columns=['Hour']
df_test=df_test.drop(labels=['Hour'],axis=1)
df_test=pd.concat([Hour_test,df_test],axis=1)
"""

"\n#le = preprocessing.LabelEncoder()\nle_res=le.fit_transform(df['Hour'])\nHour=pd.DataFrame(le_res)\n\n#District=pd.get_dummies(df['PdDistrict'],drop_first=True)\nHour.columns=['Hour']\ndf=df.drop(labels=['Hour'],axis=1)\ndf=pd.concat([Hour,df],axis=1)\n\n\n#le = preprocessing.LabelEncoder()\nle_res_test=le.fit_transform(df_test['Hour'])\nHour_test=pd.DataFrame(le_res_test)\n\n#District=pd.get_dummies(df['PdDistrict'],drop_first=True)\nHour_test.columns=['Hour']\ndf_test=df_test.drop(labels=['Hour'],axis=1)\ndf_test=pd.concat([Hour_test,df_test],axis=1)\n"

Label encoding: Minute

In [ ]:
"""
le_res=le.fit_transform(df['Minute'])
Minute=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Minute.columns=['Minute']
df=df.drop(labels=['Minute'],axis=1)
df=pd.concat([Minute,df],axis=1)


le_res_test=le.fit_transform(df_test['Minute'])
Minute_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Minute_test.columns=['Minute']
df_test=df_test.drop(labels=['Minute'],axis=1)
df_test=pd.concat([Minute_test,df_test],axis=1)

"""

"\nle_res=le.fit_transform(df['Minute'])\nMinute=pd.DataFrame(le_res)\n\n#District=pd.get_dummies(df['PdDistrict'],drop_first=True)\nMinute.columns=['Minute']\ndf=df.drop(labels=['Minute'],axis=1)\ndf=pd.concat([Minute,df],axis=1)\n\n\nle_res_test=le.fit_transform(df_test['Minute'])\nMinute_test=pd.DataFrame(le_res_test)\n\n#District=pd.get_dummies(df['PdDistrict'],drop_first=True)\nMinute_test.columns=['Minute']\ndf_test=df_test.drop(labels=['Minute'],axis=1)\ndf_test=pd.concat([Minute_test,df_test],axis=1)\n\n"

# One Hot Encoding

In [ ]:
df=pd.get_dummies(df,columns=['DayOfWeek','PdDistrict','Year','Month','Day','Hour','Minute'],drop_first=True)

df_test=pd.get_dummies(df_test,columns=['DayOfWeek','PdDistrict','Year','Month','Day','Hour','Minute'],drop_first=True)

In [ ]:
df_test.columns.nunique()

72

In [ ]:
df.columns.nunique()

72

In [ ]:
te=ce.HelmertEncoder(cols=['X','Y','rot45_X','rot45_Y','radial45','rot30_X','rot30_Y','radial30','rot60_X','radial60'],return_df=True)
df=te.fit_transform(df,df)

36 Crime Category are there 

# Applying different Models

In [ ]:
#Independent Column
X=df
X.shape

In [ ]:
#Dependent column
y.shape

In [ ]:
#splitting the dataset into the training set and test set
#from sklearn import model_selection as ms

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
"""from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
X_train=sc_X.fit_tranform(X_train)
X_test=sc_X.transform(X_test)"""

# XGBoost Model

In [ ]:
X.head()

In [ ]:
model = xgb.XGBClassifier(objective='multi:softprob',max_depth=9,n_estimators=80,learning_rate=0.35)
model.fit(X_train,y_train)

In [ ]:
y_pred=model.predict_proba(X_test)

In [ ]:
#y_pred.to_csv("submit.csv", float_format = '%.5F')
print (log_loss(y_test,y_pred));